In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import os, time, stat, io
import plotly

import plotly.graph_objects as go

plt.style.use('fivethirtyeight')
# # for requests
# os.environ['HTTP_PROXY'] = "http://fwdproxy:8080"
# os.environ['HTTPS_PROXY'] = "https://fwdproxy:8080"  

# # for urllib
# os.environ['http_proxy'] = "fwdproxy:8080"
# os.environ['https_proxy'] = "fwdproxy:8080"


In [2]:
df_state_map = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2011_us_ag_exports.csv')
df_state_map.head()

,code,state,category,total exports,beef,pork,poultry,dairy,fruits fresh,fruits proc,total fruits,veggies fresh,veggies proc,total veggies,corn,wheat,cotton
0,AL,Alabama,state,1390.63,34.4,10.6,481.0,4.06,8.0,17.1,25.11,5.5,8.9,14.33,34.9,70.0,317.61
1,AK,Alaska,state,13.31,0.2,0.1,0.0,0.19,0.0,0.0,0.00,0.6,1.0,1.56,0.0,0.0,0.00
2,AZ,Arizona,state,1463.17,71.3,17.9,0.0,105.48,19.3,41.0,60.27,147.5,239.4,386.91,7.3,48.7,423.95
3,AR,Arkansas,state,3586.02,53.2,29.4,562.9,3.53,2.2,4.7,6.88,4.4,7.1,11.45,69.5,114.5,665.44
4,CA,California,state,16472.88,228.7,11.1,225.4,929.95,2791.8,5944.6,8736.40,803.2,1303.5,2106.79,34.6,249.3,1064.95


In [14]:
df_hist_claims_raw = pd.read_csv('./data/r539cy.csv').dropna(how='all')
df_hist_claims_raw.tail()

,State,Filed week ended,Initial Claims,Reflecting Week Ended,Continued Claims,Covered Employment,Insured Unemployment Rate
3440,Wyoming,2/29/20,471,2/22/20,"3,543","268,518",1.32
3441,Wyoming,3/7/20,534,2/29/20,"3,508","268,518",1.31
3442,Wyoming,3/14/20,517,3/7/20,"3,375","268,518",1.26
3443,Wyoming,3/21/20,"3,653",3/14/20,"3,197","268,518",1.19
3444,Wyoming,3/28/20,"6,396",3/21/20,"4,519","268,518",1.68


In [15]:
df_hist_claims = pd.merge(df_hist_claims_raw, df_state_map[['code','state']],
                     left_on='State', right_on='state')

df_hist_claims['dt'] = pd.to_datetime(df_hist_claims['Filed week ended'])
df_hist_claims = df_hist_claims.set_index(['code','dt'])
df_hist_claims['init_claims'] = df_hist_claims['Initial Claims'].str.replace(',','').astype(float)
df_hist_claims.head()

State Filed week ended Initial Claims  \
code dt                                                    
AL   2019-01-05  Alabama           1/5/19          6,660   
     2019-01-12  Alabama          1/12/19          4,386   
     2019-01-19  Alabama          1/19/19          3,217   
     2019-01-26  Alabama          1/26/19          2,414   
     2019-02-02  Alabama           2/2/19          2,505   

                Reflecting Week Ended Continued Claims Covered Employment  \
code dt                                                                     
AL   2019-01-05              12/29/18           22,085          1,894,608   
     2019-01-12                1/5/19           20,075          1,894,608   
     2019-01-19               1/12/19           17,574          1,894,608   
     2019-01-26               1/19/19           16,904          1,894,608   
     2019-02-02               1/26/19           16,343          1,894,608   

                 Insured Unemployment Rate    state  init_claims  
code dt                                                           
AL   2019-01-05                       1.17  Alabama       6660.0  
     2019-01-12                       1.06  Alabama       4386.0  
     2019-01-19                       0.93  Alabama       3217.0  
     2019-01-26                       0.89  Alabama       2414.0  
     2019-02-02                       0.86  Alabama       2505.0

## Week ending Apr 4

In [16]:
df_claims_0404_raw = pd.read_csv('./data/claims_20200409.csv').iloc[:,[0,1]]
df_claims_0404_raw.columns = ['State', 'init_claims']
df_claims_0404_raw['State'] = df_claims_0404_raw.State.str.replace('*','')
df_claims_0404 = pd.merge(df_claims_0404_raw, df_state_map[['code','state']], left_on='State', right_on='state')
df_claims_0404 = df_claims_0404.set_index(['code'])
df_claims_0404['init_claims'] = df_claims_0404['init_claims'].str.replace(',','').astype(float)
df_claims_0404['dt'] = pd.Timestamp('2020-04-04')
df_claims_0404 = df_claims_0404.reset_index()[['code','state','dt','init_claims']].set_index(['code','dt'])
df_claims_0404.head()
# pd.merge(df_claims_0404_raw, df_state_map[['code','STATE']], left_on='STATE', right_on='state')

,,state,init_claims
code,dt,,
AL,2020-04-04,Alabama,105607.0
AK,2020-04-04,Alaska,15370.0
AZ,2020-04-04,Arizona,132189.0
AR,2020-04-04,Arkansas,60992.0
CA,2020-04-04,California,925450.0


# Week ending Mar 28

In [8]:
df_claims_0328_raw = pd.read_csv('./data/claims_20200402.csv').iloc[:,[0,1]]
df_claims_0328_raw.columns = ['State', 'init_claims']
df_claims_0328_raw['State'] = df_claims_0328_raw.State.str.replace('*','')
df_claims_0328 = pd.merge(df_claims_0328_raw, df_state_map[['code','state']], left_on='State', right_on='state')
df_claims_0328 = df_claims_0328.set_index(['code'])
df_claims_0328['init_claims'] = df_claims_0328['init_claims'].str.replace(',','').astype(float)
df_claims_0328['dt'] = pd.Timestamp('2020-03-28')
df_claims_0328 = df_claims_0328.reset_index()[['code','state','dt','init_claims']].set_index(['code','dt'])
df_claims_0328.head()
# pd.merge(df_claims_0328_raw, df_state_map[['code','STATE']], left_on='STATE', right_on='state')

,,state,init_claims
code,dt,,
AL,2020-03-28,Alabama,80186.0
AK,2020-03-28,Alaska,14523.0
AZ,2020-03-28,Arizona,89064.0
AR,2020-03-28,Arkansas,26944.0
CA,2020-03-28,California,878727.0


# Mar 21

In [46]:
raw_current = """Alabama 9,490 1,819 7,671 11,070 13,070 -2,000
Alaska 8,225 1,120 7,105 8,407 8,522 -115
Arizona 29,268 3,844 25,424 16,076 17,595 -1,519
Arkansas 8,958 1,382 7,576 9,851 11,514 -1,663
California 186,809 57,606 129,203 430,335 351,344 78,991
Colorado 19,429 2,321 17,108 21,987 21,260 727
Connecticut 25,098 3,440 21,658 45,284 39,786 5,498
Delaware 10,720 472 10,248 5,627 5,310 317
District of Columbia 13,473 1,213 12,260 7,093 6,746 347
Florida 74,021 6,463 67,558 29,612 32,244 -2,632
Georgia 11,746 5,445 6,301 26,644 25,352 1,292
Hawaii 8,904 1,589 7,315 6,700 6,171 529
Idaho 13,314 1,031 12,283 7,562 9,336 -1,774
Illinois 114,663 10,870 103,793 125,914 130,325 -4,411
Indiana 61,635 2,596 59,039 22,440 21,473 967
Iowa 41,890 2,229 39,661 27,172 27,816 -644
Kansas 23,687 1,755 21,932 10,245 9,771 474
Kentucky 48,847 2,785 46,062 21,885 22,940 -1,055
Louisiana 72,620 2,255 70,365 12,052 13,987 -1,935
Maine 21,197 634 20,563 8,620 8,451 169
Maryland 41,882 3,864 38,018 26,072 27,046 -974
Massachusetts 147,995 7,449 140,546 82,475 74,336 8,139
Michigan 129,298 5,338 123,960 78,824 75,757 3,067
Minnesota 116,438 4,010 112,428 70,530 61,781 8,749
Mississippi 6,723 1,147 5,576 5,714 7,098 -1,384
Missouri 40,508 4,016 36,492 22,188 22,381 -193
Montana 14,704 817 13,887 10,054 10,112 -58
Nebraska 15,668 795 14,873 4,603 5,076 -473
Nevada 93,036 6,356 86,680 20,852 19,475 1,377
New Hampshire 21,878 642 21236 3,731 3,909 -178
New Jersey 155,454 9,467 145,987 108,091 104,283 3,808
New Mexico 17,187 869 16,318 9,707 9,566 141
New York 80,334 14,272 66,062 168,921 158,268 10,653
North Carolina 93,587 3,533 90,054 19,377 19,683 -306
North Dakota 5,968 415 5,553 7,165 6,349 816
Ohio* 187,784 7,046 180,738 68,743 68,516 227
Oklahoma 17,720 1,836 15,884 15,863 16,482 -619
Oregon 22,824 4,269 18,555 30,488 29,002 1,486
Pennsylvania* 378,908 15,439 363,469 128,932 128,842 90
Puerto Rico* 1,471 1,172 299 17,198 18,704 -1,506
Rhode Island 35,436 1,108 34,328 11,011 10,692 319
South Carolina 31,064 2,093 28,971 14,780 15,032 -252
South Dakota 1,703 190 1513 2,657 2,870 -213
Tennessee 39,096 2,702 36,394 17,235 18,450 -1,215
Texas 155,657 16,176 139,481 126,955 127,905 -950
Utah 1,314 1,305 9 10,324 10,701 -377
Vermont 3,667 659 3,008 5,055 4,896 159
Virgin Islands 58 44 14 588 592 -4
Virginia 46,885 2,706 44,179 22,291 21,336 955
Washington 133,478 14,240 119,238 67,415 55,588 11,827
West Virginia 3,435 865 2,570 16,571 15,394 1,177
Wisconsin 50,957 5,190 45,767 44,278 40,738 3,540
Wyoming 2,339 517 1,822 3,929 3,375 554
US Total 2,898,450 251,416 2,647,034 2,097,193 1,977,248 119,945"""

In [47]:
def get_name(row_text):
#     text = 'West Virginia 3,435 865 2,570 16,571 15,394 1,177' #"Alabama 9,490 1,819 7,671 11,070 13,070 -2,000"

    splitted = []   # our result
    tmp = []        # our temporary character collector
    tmp_digit = []

    for c in row_text:
        if not c.isdigit() and not c == ' ' and not c == ','  and not c == '-' and not c == '*':
            tmp.append(c)    # not a digit, add it

        elif tmp:            # c is a digit, if tmp filled, add it  
            splitted.append(''.join(tmp))
            tmp = []

    if tmp:
        splitted.append(''.join(tmp))

    return(' '.join(splitted))

def get_nums(row_text):
#     text = 'West Virginia 3,435 865 2,570 16,571 15,394 1,177' #"Alabama 9,490 1,819 7,671 11,070 13,070 -2,000"

    splitted = []   # our result
    tmp = []        # our temporary character collector

    for c in row_text:
        if c.isdigit()  or c == ','  or c == '-':
            tmp.append(c)    # not a digit, add it

        elif tmp:            # c is a digit, if tmp filled, add it  
            splitted.append(''.join(tmp))
            tmp = []

    if tmp:
        splitted.append(''.join(tmp))

    return((splitted))


def runboth(row_text):
    state = get_name(row_text)
    nums = get_nums(row_text)
    return state, nums



In [48]:
new_dict = {}

for line in raw_current.split('\n'):
    state, nums = runboth(line)
    new_dict[state] = nums

new_claims_raw = pd.DataFrame(new_dict)

df_new_claims = new_claims_raw.T.rename(columns={0:'initclaims_0321',
                                 1:'initclaims_0314',
                                 2:'initclaims_chg',
                                 3:'insurunemp_0314',
                                 4:'insurunemp_0307',
                                 5:'insurunemp_cng'
                                })
for col in df_new_claims.columns:
    df_new_claims[col] = df_new_claims[col].str.replace(',','').astype(float)
    
df_new_claims['iniclaims_pctchg'] = (df_new_claims['initclaims_0321']
                                     / df_new_claims['initclaims_0314'] - 1) * 100.

df_new_claims = pd.merge(df_new_claims.reset_index(), df_state_map[['code','state']], left_on='index', right_on='state')
df_new_claims.head()

,index,initclaims_0321,initclaims_0314,initclaims_chg,insurunemp_0314,insurunemp_0307,insurunemp_cng,iniclaims_pctchg,code,state
0,Alabama,9490.0,1819.0,7671.0,11070.0,13070.0,-2000.0,421.715228,AL,Alabama
1,Alaska,8225.0,1120.0,7105.0,8407.0,8522.0,-115.0,634.375000,AK,Alaska
2,Arizona,29268.0,3844.0,25424.0,16076.0,17595.0,-1519.0,661.394381,AZ,Arizona
3,Arkansas,8958.0,1382.0,7576.0,9851.0,11514.0,-1663.0,548.191027,AR,Arkansas
4,California,186809.0,57606.0,129203.0,430335.0,351344.0,78991.0,224.287401,CA,California


In [65]:
df_claims_raw = pd.read_csv('./r539cy.csv')
df_claims_raw['Filed week ended'].dropna().sort_values()


2485    1/11/20
629     1/11/20
1525    1/11/20
1205    1/11/20
2293    1/11/20
         ...   
995      9/7/19
3107     9/7/19
1507     9/7/19
2467     9/7/19
2339     9/7/19
Name: Filed week ended, Length: 3392, dtype: object

In [70]:
df_claims = pd.merge(df_claims_raw, df_state_map[['code','state']], left_on='State', right_on='state')

df_claims['dt'] = pd.to_datetime(df_claims['Filed week ended'])
df_claims = df_claims.set_index(['code','dt'])
df_claims['init_claims'] = df_claims['Initial Claims'].str.replace(',','').astype(float)
df_claims.head()

State Filed week ended Initial Claims  \
code dt                                                    
AL   2019-01-05  Alabama           1/5/19          6,660   
     2019-01-12  Alabama          1/12/19          4,386   
     2019-01-19  Alabama          1/19/19          3,217   
     2019-01-26  Alabama          1/26/19          2,414   
     2019-02-02  Alabama           2/2/19          2,505   

                Reflecting Week Ended Continued Claims Covered Employment  \
code dt                                                                     
AL   2019-01-05              12/29/18           22,085          1,894,608   
     2019-01-12                1/5/19           20,075          1,894,608   
     2019-01-19               1/12/19           17,574          1,894,608   
     2019-01-26               1/19/19           16,904          1,894,608   
     2019-02-02               1/26/19           16,343          1,894,608   

                 Insured Unemployment Rate    state  init_claims  
code dt                                                           
AL   2019-01-05                       1.17  Alabama       6660.0  
     2019-01-12                       1.06  Alabama       4386.0  
     2019-01-19                       0.93  Alabama       3217.0  
     2019-01-26                       0.89  Alabama       2414.0  
     2019-02-02                       0.86  Alabama       2505.0

In [86]:
# df_init_claims = df_claims.init_claims.unstack(0)
# df_claims['init_claims'].unstack(0).loc['2020-03-21']

# df_chart = df_claims['init_claims'].unstack(0).loc['2020-03-21'] #df_new_claims#df_init_claims.pct_change().iloc[-1]*100.

# fig = go.Figure(data=go.Choropleth(
#     locations=df_chart.code, # Spatial coordinates
#     z = df_chart['initclaims_0321'], # Data to be color-coded
#     locationmode = 'USA-states', # set of locations match entries in `locations`
#     colorscale = 'Reds',
#     colorbar_title = "Init. Unemp. Claims",
#     zmax = df_claims_0328.init_claims.max(),
#     zmin = 0.
# ))

# fig.update_layout(
#     title_text = 'Initial Unemployment Claims for the week ending Mar 21 2020',
#     geo_scope='usa', # limite map scope to USA
# )

# fig.show()

# df_chart = df_new_claims#df_init_claims.pct_change().iloc[-1]*100.

# fig = go.Figure(data=go.Choropleth(
#     locations=df_chart.code, # Spatial coordinates
#     z = df_chart['iniclaims_pctchg'], # Data to be color-coded
#     locationmode = 'USA-states', # set of locations match entries in `locations`
#     colorscale = 'Reds',
#     colorbar_title = "% Change in Init. Unemp. Claims"
# ))

# fig.update_layout(
#     title_text = 'Weekly % Change in Initial Unemployment Claims for the week ending Mar 21 2020',
#     geo_scope='usa', # limite map scope to USA
# )

# fig.show()

# Charts

In [17]:
df_hist_claims_new = pd.concat([df_hist_claims,df_claims_0404])
df_hist_claims_new.tail()

,,State,Filed week ended,Initial Claims,Reflecting Week Ended,Continued Claims,Covered Employment,Insured Unemployment Rate,state,init_claims
code,dt,,,,,,,,,
VA,2020-04-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Virginia,149758.0
WA,2020-04-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Washington,176827.0
WV,2020-04-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,West Virginia,14145.0
WI,2020-04-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Wisconsin,104776.0
WY,2020-04-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Wyoming,4900.0


## Current week

In [25]:
this_wk_dt = '2020-04-04'
last_wk_dt = '2020-03-28'
this_wk_str = 'Apr 4 2020'
last_wk_str = 'Mar 28 2020'

In [18]:
df_initclaims = df_hist_claims_new['init_claims'].unstack(0)
df_initclaims.tail()

code,AK,AL,AR,AZ,CA,CO,CT,DE,FL,GA,...,SD,TN,TX,UT,VA,VT,WA,WI,WV,WY
dt,,,,,,,,,,,,,,,,,,,,,
2020-03-07,815.0,1663.0,1843.0,3357.0,43385.0,1865.0,2527.0,518.0,5325.0,4569.0,...,185.0,2032.0,14355.0,1015.0,2527.0,446.0,6616.0,5052.0,1033.0,534.0
2020-03-14,1120.0,1819.0,1382.0,3844.0,57606.0,2321.0,3440.0,472.0,6463.0,5445.0,...,190.0,2702.0,16176.0,1305.0,2706.0,659.0,14240.0,5190.0,865.0,517.0
2020-03-21,7847.0,10892.0,9275.0,29348.0,186333.0,19774.0,25100.0,10776.0,74313.0,12140.0,...,1761.0,38077.0,155426.0,19690.0,46277.0,3784.0,129909.0,51031.0,3536.0,3653.0
2020-03-28,13774.0,80984.0,27756.0,88940.0,1058325.0,61838.0,33227.0,19137.0,228484.0,133820.0,...,6801.0,92500.0,276185.0,28532.0,112497.0,14633.0,182849.0,110934.0,14523.0,6396.0
2020-04-04,15370.0,105607.0,60992.0,132189.0,925450.0,45494.0,33418.0,18863.0,169885.0,388175.0,...,7916.0,116141.0,313832.0,33076.0,149758.0,16176.0,176827.0,104776.0,14145.0,4900.0


In [20]:
df_initclaims.loc[this_wk_dt].max()

925450.0

In [32]:
def ch_init_claims(df_chart, chart_title, cb_title, zrange=False, mean_zero=False):
    
    if zrange:
        zmin, zmax = zrange
    else:
        zmin, zmax = df_chart.min(), df_chart.max()
        
    if mean_zero:
        zmin = zmax*-1
        cscale = 'picnic'
    else:
        zmin = df_chart.min()
        cscale = 'Reds'

    fig = go.Figure(data=go.Choropleth(
        locations=df_chart.index, # Spatial coordinates
        z = df_chart, # Data to be color-coded
        locationmode = 'USA-states', # set of locations match entries in `locations`
        colorscale = cscale,
        colorbar_title = cb_title,
        zmax = zmax, zmin=zmin
    ))

    fig.update_layout(
        title_text = chart_title,
        geo_scope='usa', # limite map scope to USA
    )

    fig.show()

In [33]:
ch_init_claims(df_initclaims.loc[this_wk_dt],
               'Initial Unemployment Claims for the week ending {}'.format(this_wk_str),
               "Init. Unemp. Claims"
              )
ch_init_claims(df_initclaims.loc[last_wk_dt],
               'Initial Unemployment Claims for the week ending {}'.format(last_wk_str),
               "Init. Unemp. Claims", (df_initclaims.loc[this_wk_dt].min(),
                                       df_initclaims.loc[this_wk_dt].max())
              )

In [34]:
ch_init_claims(df_initclaims.pct_change().mul(100).loc[this_wk_dt],
               '% Change from prev. week in Init. Unemp. Claims for the week of {}'.format(this_wk_str),
               "% Change in Init. Unemp. Claims",
              False, True)
ch_init_claims(df_initclaims.pct_change().mul(100).loc[last_wk_dt],
               '% Change from prev. week in Init. Unemp. Claims for the week of {}'.format(last_wk_str),
               "% Change in Init. Unemp. Claims", 
               (df_initclaims.pct_change().mul(100).loc[this_wk_dt].min(),
                df_initclaims.pct_change().mul(100).loc[this_wk_dt].max()), True
              )

In [36]:
ch_init_claims(df_initclaims.pct_change(52).mul(100).loc[this_wk_dt],
               '% Change from Last Year in Init. Unemp. Claims for the week of {}'.format(this_wk_str),
               "% Change in Init. Unemp. Claims",
              False, False)

ch_init_claims(df_initclaims.pct_change(52).mul(100).loc[last_wk_dt],
               '% Change from Last Year in Init. Unemp. Claims for the week of {}'.format(last_wk_str),
               "% Change in Init. Unemp. Claims", 
               (df_initclaims.pct_change(52).mul(100).loc[this_wk_dt].min(),
                df_initclaims.pct_change(52).mul(100).loc[this_wk_dt].max()), False
              )